# SETUP

In [ ]:
;;
#require "pkp"

In [ ]:
open Pkp

open Gp

module A = Balanced_net

In [ ]:
let dt = 1E-3

let n_neurons = 1000

let rx = 5.0

In [ ]:
let plot_raster ?(ps = 0.2) raster color (module P : Plot) =
  P.plot
    (L [ Mat.col raster 0; Mat.col raster 1 ])
    ~style:(Printf.sprintf "p pt 7 lc rgb '%s' ps %f" color ps)
    [ barebone
    ; ylabel "neurons"
    ; yrange (0.0, 100.0)
    ; set
        "object 1 rectangle from first 0, graph -0.1 rto first 0.2, graph 0.05 fs solid \
         1.0 fc rgb 'black' noclip"
    ]

----
## A GROUP OF "RANDOM SPIKERS"

In [ ]:
module X = A.Poisson_population (struct
  let n = n_neurons
  and rate = rx
  and dt = dt
end)

In [ ]:
let _ =
  let d = 3. in
  let raster =
    Array.init
      (int_of_float (d /. dt))
      (fun t ->
        X.step ();
        X.spikes ())
    |> A.plottable_raster ~dt
  in
  Juplot.draw ~fmt:`svg ~size:(600, 300) (plot_raster ~ps:0.5 raster "black")

-----
# A SINGLE NEURON receving input from the "random spikers"

In [ ]:
module S = A.LIF_population (struct
  let n = 1
  and dt = dt
  and free_vm = false
  and tau = 20E-3
  and threshold = 1.0
end)

In [ ]:
let _ =
  let duration = 10. in
  let module Prms = struct
    let n = n_neurons
    let rate = rx
    let dt = dt
  end
  in
  let module XE = A.Poisson_population (Prms) in
  let module XI = A.Poisson_population (Prms) in
  (* strength of the connections *)
  let w = 1.8 /. sqrt (float n_neurons) in
  (* 1.8 *)
  (* input: this LIF neuron receives input from ALL neurons in XE and XI *)
  let inputs =
    [ +.w, A.all_to_all_synapses 1 n_neurons, XE.spikes
    ; -.w, A.all_to_all_synapses 1 n_neurons, XI.spikes
    ]
  in
  let n_bins = int_of_float (duration /. dt) in
  let count = ref 0 in
  let voltage =
    Mat.init 1 n_bins (fun t ->
        XE.step ();
        XI.step ();
        S.step inputs;
        if Array.length (S.spikes ()) > 0
        then (
          incr count;
          5.)
        else Mat.(get S.us 0 0))
  in
  let plot (module P : Plot) = P.plot (A voltage) ~style:"l lc 8 lw 2" [ barebone ] in
  Juplot.draw ~fmt:`svg ~size:(1000, 120) plot;
  float !count /. duration

----
## BIG RECURRENT NETWORK!

In [ ]:
module Prms = struct
  let n = n_neurons
  and dt = dt
  and free_vm = false
  and tau = 20E-3
  and threshold = 1.0
end

module E = A.LIF_population (Prms)

module I = A.LIF_population (Prms)

In [ ]:
let k = 100

let z = 1. /. sqrt (float k)

let inputs jx je ji =
  [ z *. jx, A.random_synapses n_neurons n_neurons k, X.spikes
  ; z *. je, A.random_synapses n_neurons n_neurons k, E.spikes
  ; z *. ji, A.random_synapses n_neurons n_neurons k, I.spikes
  ]

let inputs_e = inputs 1. 1. (-2.)

let inputs_i = inputs 0.9 1.5 (-2.)

In [ ]:
let _ =
  let progress = Pkp.Misc.time_indicator dt in
  let d = 3. in
  let rasters =
    Array.init
      (int_of_float (d /. dt))
      (fun t ->
        if t mod 10 = 0 then progress t;
        X.step ();
        E.step inputs_e;
        I.step inputs_i;
        X.spikes (), E.spikes (), I.spikes ())
  in
  let raster extract = rasters |> Array.map extract |> A.plottable_raster ~dt in
  Juplot.draw ~fmt:`svg ~size:(1000, 120) (fun (module P : Plot) ->
      P.multiplot (1, 3) (fun i j k ->
          let extract (a, b, c) = if i = 0 then a else if i = 1 then b else c in
          let color = if i = 0 then "black" else if i = 1 then "red" else "blue" in
          plot_raster (raster extract) color (module P)))

Mathematical calculations show that the firing rates $r_E$ and $r_I$ should solve:
$$ 1.0 \times r_E -2 \times r_I + 1.0 \times r_X = 0$$
$$ 1.5 \times r_E -2 \times r_I + 0.9 \times r_X = 0 $$

---